In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/cleaned-youtube-desk-setup.csv')
df.head()

,video_id,title,channel,release_date,transcript_text,word_count
0,gYgaxdUAAS0,My NEW Ultimate Desk &amp; Setup Tour 2023!,Techno Tim,2023-10-21 15:00:58,What's up everyone? How's it going? So this pa...,3674
1,8cZtbjI5FnQ,My NEW Ultimate Desk Setup &amp; Office Tour 2...,UrAvgConsumer,2023-09-30 16:21:44,what's going on guys it's your average consume...,2856
2,D43lCHSdrSo,The ALL-ROUNDER Desk Setup - Build &amp; Tour ...,ThisIsE,2023-10-21 13:18:22,this video is brought to you by ug green I've ...,2528
3,KwW-m69tAz8,My ultimate DREAM desk setup (2023),Patrick Rambles,2023-07-04 13:00:13,so guys ever since I've moved into this new st...,3168
4,SjcvBFfMulY,My Productivity Desk Setup (2023),Ali Abdaal,2023-09-19 13:00:32,foreign welcome back to the channel and to m...,4503


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   video_id         770 non-null    object
 1   title            770 non-null    object
 2   channel          770 non-null    object
 3   release_date     770 non-null    object
 4   transcript_text  770 non-null    object
 5   word_count       770 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 36.2+ KB


In [4]:
df['release_date'] = pd.to_datetime(df['release_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   video_id         770 non-null    object        
 1   title            770 non-null    object        
 2   channel          770 non-null    object        
 3   release_date     770 non-null    datetime64[ns]
 4   transcript_text  770 non-null    object        
 5   word_count       770 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 36.2+ KB


In [5]:
df.head()

,video_id,title,channel,release_date,transcript_text,word_count
0,gYgaxdUAAS0,My NEW Ultimate Desk &amp; Setup Tour 2023!,Techno Tim,2023-10-21 15:00:58,What's up everyone? How's it going? So this pa...,3674
1,8cZtbjI5FnQ,My NEW Ultimate Desk Setup &amp; Office Tour 2...,UrAvgConsumer,2023-09-30 16:21:44,what's going on guys it's your average consume...,2856
2,D43lCHSdrSo,The ALL-ROUNDER Desk Setup - Build &amp; Tour ...,ThisIsE,2023-10-21 13:18:22,this video is brought to you by ug green I've ...,2528
3,KwW-m69tAz8,My ultimate DREAM desk setup (2023),Patrick Rambles,2023-07-04 13:00:13,so guys ever since I've moved into this new st...,3168
4,SjcvBFfMulY,My Productivity Desk Setup (2023),Ali Abdaal,2023-09-19 13:00:32,foreign welcome back to the channel and to m...,4503
